## Beta- Each Ticker to Portfolio/Fund - Rolling selection

### import packages

In [2]:
import ffn
import warnings
import numpy as np
import pandas as pd
import statistics as st
pd.set_option('display.float_format',lambda x : '%.7f' % x)
warnings.filterwarnings("ignore")

### set weight

In [2]:
ticker_dict = {}
ticker_dict["ben"]= 0.395342633
ticker_dict["alxn"]= 0.165616348
ticker_dict["twtr"]= 0.080393952
ticker_dict["fslr"] = 0.057603788
ticker_dict["bidu"] = 0.051790624
ticker_dict["axp"] = 0.050945364
ticker_dict["etn"] = 0.048031878
ticker_dict["amgn"] = 0.03021901
ticker_dict["aapl"] = 0.03028385
ticker_dict["enph"] = 0.021541526
ticker_dict["fb"] = 0.018584452
ticker_dict["baba"] = 0.015565296
ticker_dict["sedg"] = 0.011624454
ticker_dict["nflx"] = 0.006921963
ticker_dict["nvda"] = 0.006137999
ticker_dict["tsla"] = 0.005327053
ticker_dict["goog"] = 0.002887009
ticker_dict["amzn"] = 0.001182802

### Set ticker and time

In [3]:
ticker_lists = "NFLX,NVDA,BABA,BIDU,TWTR,TSLA,AAPL,AMZN,ENPH,FB,GOOG,FSLR,SEDG,AXP,BEN,ALXN,AMGN,ETN"
ranges = ticker_lists.split(",")
start_date = "2015-2-20"



### Calculation Beta

In [4]:
while len(ranges) > 1:
    portfolio = ffn.get(ranges, start = start_date)
    
     
    # Create Fund/portfolio with existing weight
    portfolio["Fund"] = 0
    for i in ranges:    
        i = i.lower()
        portfolio["Fund"] += (portfolio[i] * ticker_dict[i] )
    
    
    portfolio_stats = portfolio.calc_stats()
    
    #return correlation
    corr = portfolio.pct_change().corr()  
    
    #vol fraction
    Beta_table = portfolio_stats.stats.loc[["daily_vol"], :]
    returns = portfolio["Fund"].pct_change()
    Beta_table["Fund"] = (returns.std()) * np.sqrt(252)  #annualize volitility
    Beta_table = Beta_table.T
    Beta_table["D"] =  Beta_table.loc["Fund"]['daily_vol']  #create Denominator
    Beta_table["frac"] = Beta_table["daily_vol"]/Beta_table["D"] #create volitility porpotion
    
    
    #combine to correlation matrix
    corr["frac"] = Beta_table["frac"]
    Fund = corr["Fund"] *  corr["frac"]
    
    #print(Fund,"\n")
    
    
    # Selection methodology
    Fund = Fund.loc[Fund != Fund["Fund"]]
    max_beta = Fund.max()
    Fund_ticker = Fund.loc[Fund!=abs(Fund.max())]
    
    # update ranges
    ranges = []
    for i in Fund_ticker.index:   
        ranges.append(str(i))
        
    
    max_ticker = Fund.loc[Fund == max_beta] 
    
    # locate annual return of porrtfolio
    Annual_return = portfolio_stats.stats
    Annual_return = pd.DataFrame(Annual_return)
    Annual_return = Annual_return.T
    Annual_return = Annual_return["one_year"]["Fund"]
    
    
    print("Annual_Return:%s" %Annual_return,"\n", "dayily_vol:%s" %Beta_table.loc["Fund"]['daily_vol'],"\n", "Max beta is:%s" %max_ticker,"\n", "Adjusted ticker List:%s" %ranges ,"\n""\n")

Annual_Return:0.5797063200843735 
 dayily_vol:0.24193539505919082 
 Max beta is:alxn   1.2901563
dtype: object 
 Adjusted ticker List:['nflx', 'nvda', 'baba', 'bidu', 'twtr', 'tsla', 'aapl', 'amzn', 'enph', 'fb', 'goog', 'fslr', 'sedg', 'axp', 'ben', 'amgn', 'etn'] 


Annual_Return:0.5834303210620642 
 dayily_vol:0.23002254815883386 
 Max beta is:enph   1.4541007
dtype: object 
 Adjusted ticker List:['nflx', 'nvda', 'baba', 'bidu', 'twtr', 'tsla', 'aapl', 'amzn', 'fb', 'goog', 'fslr', 'sedg', 'axp', 'ben', 'amgn', 'etn'] 


Annual_Return:0.5505461874269011 
 dayily_vol:0.22851523017157477 
 Max beta is:sedg   1.2426233
dtype: object 
 Adjusted ticker List:['nflx', 'nvda', 'baba', 'bidu', 'twtr', 'tsla', 'aapl', 'amzn', 'fb', 'goog', 'fslr', 'axp', 'ben', 'amgn', 'etn'] 


Annual_Return:0.5318440223703629 
 dayily_vol:0.2264416263691445 
 Max beta is:nvda   1.2240646
dtype: object 
 Adjusted ticker List:['nflx', 'baba', 'bidu', 'twtr', 'tsla', 'aapl', 'amzn', 'fb', 'goog', 'fslr', 'axp'

## Beta- Ticker to S&P 500

### Set ticker + S&P

In [5]:
ticker_lists_sp = "NFLX,NVDA,BABA,BIDU,TWTR,TSLA,AAPL,AMZN,ENPH,FB,GOOG,FSLR,SEDG,AXP,BEN,ALXN,AMGN,ETN,^GSPC"
ranges_sp = ticker_lists_sp.split(",")
start_date = "2015-2-20"

### Calculate beta

In [6]:
portfolio_sp = ffn.get(ranges_sp, start = start_date)
portfolio_sp_stats = portfolio_sp.calc_stats()

corr_sp = portfolio_sp.pct_change().corr()["gspc"]


Beta_table_sp = portfolio_sp_stats.stats.loc[["daily_vol"], :]

Beta_table_sp = Beta_table_sp.T
Beta_table_sp["D"] =  Beta_table_sp.iloc[-1,0]  #create Denominator
Beta_table_sp["frac"] = Beta_table_sp["daily_vol"]/Beta_table_sp["D"] #create volitility porpotion

Fund_sp_total = pd.merge(corr_sp, Beta_table_sp["frac"], left_index=True, right_index=True)
Fund_sp_total["beta"] = (Fund_sp_total["gspc"]*Fund_sp_total["frac"])

print(Fund_sp_total["beta"])



nflx   1.0380578
nvda   1.4798336
baba   0.9087553
bidu   0.9833671
twtr   1.2049169
tsla   1.2665978
aapl   1.1786025
amzn   0.9644161
enph   1.5327414
fb     1.0804713
goog   1.0678727
fslr   1.0795190
sedg   1.3373298
axp    1.2577915
ben    1.1489540
alxn   0.9975340
amgn   0.9135000
etn    1.1969658
gspc   1.0000000
Name: beta, dtype: object


### s&p 500 check

In [3]:
test = "NFLX,NVDA,BABA,BIDU,TWTR,TSLA,AAPL,AMZN,ENPH,FB,GOOG,FSLR,SEDG,AXP,BEN,ALXN,AMGN,ETN"
r = test.split(",")
start_date = "2015-2-20"
p = ffn.get(r, start = start_date)
p = p.calc_stats()
p.display()

Stat                 nflx        nvda        baba        bidu        twtr        tsla        aapl        amzn        enph        fb          goog        fslr        sedg        axp         ben         alxn        amgn        etn
-------------------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------
Start                2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26  2015-03-26
End                  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24  2021-02-24
Risk-free rate       0.00%       0.00%       0.00%       0.00% 